In [ ]:
import ee

ee.Authenticate() 
ee.Initialize(project='raman-461708')

FARM_CLASSES = [8, 9, 10, 11, 13]
SCRUB_CLASSES = [12, 7]
FOREST_CLASS  = [6]
PLANTATION    = [13]


lulc = ee.Image("projects/raman-461708/assets/LULC_v4_PanIndia_2023-07-01_2024-06-30").select("predicted_label").rename('lulc').toInt()
boundaries = ee.FeatureCollection("projects/raman-461708/assets/Area_UbiCherala_boundaries_0").filter(ee.Filter.neq("class", "plantation"))

def get_distribution(feature):
    feature = ee.Feature(feature)
    hist = lulc.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=feature.geometry(),
        scale=10)
    
    # 2) Pull {class_str: count} from the 'lulc' band
    counts = ee.Dictionary(hist.get('lulc', ee.Dictionary({})))

    # 3) Total pixels inside this boundary
    total = ee.Number(
        ee.List(counts.values()).iterate(
            lambda v, acc: ee.Number(acc).add(ee.Number(v)),
            0
        )
    )

    # 4) Sum helper for a list of numeric class IDs (convert each to string key)
    def sum_classes(class_list):
        class_list = ee.List(class_list)
        def _acc(c, acc):
            key = ee.Number(c).format()                  # convert 8 -> "8"
            val = ee.Number(counts.get(key, 0))          # counts have string keys
            return ee.Number(acc).add(val)
        return ee.Number(class_list.iterate(_acc, 0))

    # 5) Compute farm % = classes 8,9,10,11 / total
    farm_count = sum_classes(FARM_CLASSES)
    plantation_count = sum_classes(PLANTATION)
    
    farm_pct = ee.Algorithms.If(ee.Number(total).gt(0), ee.Number(farm_count).divide(total).multiply(100), 0)
    plantation_pct = ee.Algorithms.If(ee.Number(total).gt(0), ee.Number(plantation_count).divide(total).multiply(100), 0)
    
    return feature.set('farm_pct', farm_pct)

def simplify_boundary(feature):
    buf = 20
    tol = 5
    smooth = feature.geometry().buffer(buf).buffer(-buf+1)
    simple = smooth.simplify(tol)
    return feature.setGeometry(simple)
    
    
boundaries = boundaries.map(get_distribution)
boundaries = boundaries.boundaries.filter(ee.Filter.gt('farm_pct', 50))
boundaries = boundaries.map(simplify_boundary)

OUTPUT_ASSET = "projects/raman-461708/assets/Area_UbiCherala_boundaries_farm_plantations_GPU_Model"
task = ee.batch.Export.table.toAsset(
    collection=boundaries,
    description="Export_UbiCherala_boundaries_farm_pct",
    assetId=OUTPUT_ASSET
)

task.start()

In [8]:
import geemap
Map = geemap.Map()
url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
Map.layout.height = '1000px'
Map.add_tile_layer(url, name="Google Map", attribution="Google")

Map.addLayer(boundaries.filter(ee.Filter.gt('farm_pct', 50)), {}, 'Boundaries')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [2]:

OUTPUT_ASSET = "projects/raman-461708/assets/Area_UbiCherala_boundaries_farm_plantations_GPU_Model"
task = ee.batch.Export.table.toAsset(
    collection=boundaries.filter(ee.Filter.gt('farm_pct', 50)),
    description="Export_UbiCherala_boundaries_farm_pct",
    assetId=OUTPUT_ASSET
)

task.start()


In [3]:
boundaries = ee.FeatureCollection("projects/raman-461708/assets/Area_UbiCherala_boundaries_0").filter(ee.Filter.eq("class", "plantation"))

In [6]:
boundaries.limit(10).getInfo()

{'type': 'FeatureCollection',
 'columns': {'blueness': 'Float',
  'class': 'String',
  'ent': 'Float',
  'ent_pl': 'Float',
  'frct_dim': 'Float',
  'greeness': 'Float',
  'id': 'String',
  'num_lines': 'Float',
  'num_rt_ang': 'Float',
  'rect': 'Float',
  'red': 'Float',
  'redness': 'Float',
  'size': 'Float',
  'system:index': 'String',
  'value': 'Float'},
 'version': 1767956856146309,
 'id': 'projects/raman-461708/assets/Area_UbiCherala_boundaries_0',
 'properties': {'system:asset_size': 52354258},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[78.1718317300782, 13.99294452680446],
      [78.1718317300782, 13.992935557914732],
      [78.17190748334836, 13.99293553585981],
      [78.17190748334836, 13.992926654045345],
      [78.17196105853304, 13.99292667421949],
      [78.17196105853304, 13.992913264420226],
      [78.17201454671788, 13.992913280304853],
      [78.17201454671788, 13.992904398489127],
      [78.1720680348756, 13.9929043